In [3]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from llama_index.core import ServiceContext
sc = ServiceContext.from_defaults()

/var/folders/8l/h9gthwnn40gcs_zb4k3g4_z40000gn/T/ipykernel_50303/693524478.py:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sc = ServiceContext.from_defaults()


In [5]:
import chromadb
from llama_index.core import VectorStoreIndex, load_index_from_storage
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding

db = chromadb.PersistentClient(path="../Storing/chroma")

# collection_name = "E9_VISA_GUIDE_AND_FAQ" # Guide + FAQ
collection_name = "E9_VISA_GUIDE" # Guide
# collection_name = "E9_VISA_FAQ" # FAQ

chroma_collection = db.get_collection(collection_name)
chroma_vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

embed_model = OpenAIEmbedding()
index = VectorStoreIndex.from_vector_store(vector_store=chroma_vector_store, embed_model=embed_model)

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [6]:
retriever = index.as_retriever(
    similarity_top_k=10,
)

In [7]:
def print_nodes(nodes):
    for node in nodes:
        print(f"\nScore: {node.score:.3f}")
        print(f"Title: {node.metadata['document_title']}")
        print(node.get_content()[:100])
        print("\n--------------------------------------------------------------------\n\n")

In [96]:
# query = "베트남에서 E9 비자 발급하고 싶은데 무엇을 준비해야 되나여?" # long(X) 나머지 ok
# query = "한국에서 일하려고 하는데 주로 무슨 업종이 있나요" #cohere, long + sentence, origin
# query = "비전문취업자의 직장변경은 몇회까지 가능한가요?" # long(X) 나머지 ok
# query = "저는 베트남에서 쌀국수 식당에서 조리사로 일하고 있는데 한국에서 요리사로 취업이 가능할까요?" #origin + sentence
# query = "농축산업에서 일하려고 하는데 준비해야하는 서류는 무엇이있나요?" #cohere
# query = "고용허가제를 통해 E-9비자로 한국의 농축산업에서 일하려고 하는데 준비해야하는 서류는 무엇이있나요?" #cohere
# query = "저는 베트남에서 한국으로 취업을 희망하는 학생입니다. 한국의 농장에서 일하려고 하는데 준비해야하는 절차는 무엇이 있나요?" #cohere + long
query = "E9비자 체류기간 연장 허가와 관련해서 기숙사비 영수증은 무슨용도의 서류인가요?" # long(X) 나머지 ok
query = "E9비자와 관련해서 한국에 더 머무르는 허가를 받기위해 제출해야 하는 서류는 무엇인가요?" # cohere
query = "직장의 이름이 변경되었을때도 고용변동에 대한 신고를 해야하나요?" # long(X) 나머지 ok
query = "저는 성실근로자 재입국 취업 대상자인데 베트남으로 다시 돌아갈때 외국인등록증을 반납해야 하나요?" # cohere
query = "저는 한국에서 일하고 있는 외국인 노동자입니다. 지금 일하는 곳의 사장님이 회사이름을 변경하였는데 제가 어떤 신고를 해야될게 있나요?" # cohere

nodes = retriever.retrieve(query)

print_nodes(nodes)
# save_to_json(query, nodes, {"similarity_top_k": 10})

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"

Score: 0.748
Title: 고용허가제 해당자의 고용변동 신고
고용허가제 해당자의 고용변동 신고
        ● 신고의무자
            ○ 비전문취업(E-9) 외국인근로자를 고용한 자
        ● 신고기한
           

--------------------------------------------------------------------



Score: 0.737
Title: 고용허가제 해당자가 근무처(직장)를 변경할 수 있는 조건
고용허가제 해당자가 근무처(직장)를 변경할 수 있는 조건
        ● 출입국관리법」제21조 및 「외국인고용법」제25조에 의거, 비전문취업자의 근무처 변경에 대한 허가권자는 법

--------------------------------------------------------------------



Score: 0.718
Title: 고용허가제 해당자의 근무처(직장)의 변경
고용허가제 해당자의 근무처(직장)의 변경
        ● 출입국관리법」제21조 및 「외국인고용법」제25조에 의거, 비전문취업자의 근무처 변경에 대한 허가권자는 법무부장관임
   

--------------------------------------------------------------------



Score: 0.715
Title: 고용허가제 농업 분야 외국인근로자의 근무처(직장) 추가
고용허가제 농업 분야 외국인근로자의 근무처(직장) 추가
        ● 계절적으로 업무량의 차이가 큰 일

In [8]:
import json
def save_to_json(query, origin, cohere, long, sentence, lingua):
    origin_sources = [
        # {
        #     "rank": idx + 1,
        #     "document_title": node.metadata['document_title'],
        #     "score": round(node.score, 3),
        # }
        node.metadata['document_title'] 
        for idx, node in enumerate(origin)
    ]
    cohere_sources = [
        # {
        #     "rank": idx + 1,
        #     "document_title": node.metadata['document_title'],
        #     "score": round(node.score, 3),
        # }
        node.metadata['document_title'] 
        for idx, node in enumerate(cohere)
    ]
    long_sources = [
        # {
        #     "rank": idx + 1,
        #     "document_title": node.metadata['document_title'],
        #     "score": round(node.score, 3),
        # }
        node.metadata['document_title'] 
        for idx, node in enumerate(long)
    ]
    sentence_sources = [
        # {
        #     "rank": idx + 1,
        #     "document_title": node.metadata['document_title'],
        #     "score": round(node.score, 3),
        # }
        node.metadata['document_title'] 
        for idx, node in enumerate(sentence)
    ]
    lingua_sources = [
        # {
        #     "rank": idx + 1,
        #     "document_title": node.metadata['document_title'],
        #     "score": round(node.score, 3),
        # }
        node.metadata['document_title'] 
        for idx, node in enumerate(lingua)
    ]
    with open(f"./query_test_set/{query}.json", 'w') as json_file:
        json.dump(
            {"query": query, "origin_sources": origin_sources, "cohere_sources": cohere_sources, "long_sources": long_sources, "sentence_sources": sentence_sources, "lingua_sources": lingua_sources}, 
            json_file, 
            ensure_ascii=False, 
            indent=4
        )

In [9]:
# Install dependency.
%pip install llmlingua

Note: you may need to restart the kernel to use updated packages.


In [11]:
from llama_index.core.postprocessor import LongContextReorder
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.postprocessor import SentenceEmbeddingOptimizer
from llama_index.postprocessor.longllmlingua import LongLLMLinguaPostprocessor



cohere_postprocessor = CohereRerank(
    top_n=5, model="rerank-multilingual-v2.0"
)
long_postprocessor = LongContextReorder()
sentence_postprocessor = SentenceEmbeddingOptimizer(
    embed_model=sc.embed_model,
    percentile_cutoff=0.7,
    # threshold_cutoff=0.7
)

# lingua_postprocessor = LongLLMLinguaPostprocessor(
#     instruction_str="Given the context, please answer the final question",
#     target_token=300,
#     rank_method="longllmlingua",
#     additional_compress_kwargs={
#         "condition_compare": True,
#         "condition_in_question": "after",
#         "context_budget": "+100",
#         "reorder_context": "sort",  # enable document reorder
#     },
# )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

AssertionError: Torch not compiled with CUDA enabled

In [97]:
cohere_reorder_nodes = cohere_postprocessor.postprocess_nodes(nodes, query_str=query)
long_reorder_nodes = long_postprocessor.postprocess_nodes(nodes, query_str=query)
sentence_reorder_nodes = sentence_postprocessor.postprocess_nodes(nodes, query_str=query)

# lingua_reorder_nodes = lingua_postprocessor.postprocess_nodes(nodes, query_str=query)


# print_nodes(reorder_nodes)
save_to_json(query, nodes, cohere_reorder_nodes, long_reorder_nodes, sentence_reorder_nodes, lingua_reorder_nodes)

# reorder_nodes

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "

NameError: name 'lingua_postprocessor' is not defined